# Computing gradients and backpropagating through connectome-initialized models

While the previous tutorial showed you how to initialize a neural network with connectome-determined weights, it didn't provide you with information on how to "tune" model parameters in a data-driven manner. Turns out, computing (and backpropagating) gradients in networks with both ***sparse and dense*** tensors is non-trivial. In this tutorial, we spin up a small example on how you can accomplish this in `torch-biopl`. 

Goals:

- Continue from our previous example.
- Implement a wrapper on top of the `ConnectomeODERNN` to support a readout layer.
- Setup a simple training loop.

Note: For demonstration purposes, we'll use flattened MNIST images as inputs into the connectome. This is however simplistic and we do allow for arbitrarily complex input mappings. To learn how to do that please refer to the API.

In [1]:
import os

import torch
import torchvision.transforms as T
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from tqdm import tqdm

from bioplnn.models import ConnectomeODEClassifier

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_float32_matmul_precision("high")
print("Using device: {}".format(device))

Using device: cuda


In [3]:
# Download the connectome and read it in as a torch tensor. We have pre-processed this as a sparse tensor for the purposes of this example.
!gdown "https://drive.google.com/uc?id=18448HYpYrm60boziHG73bxN4CK5jG-1g"
connectome = torch.load("turaga-dros-visual-connectome.pt", weights_only=True)

from bioplnn.utils.torch import create_identity_ih_connectivity

# since we are feeding in MNIST images
input_size = 28 * 28
num_neurons = connectome.shape[0]

input_projection_matrix = create_identity_ih_connectivity(
    input_size=input_size,
    num_neurons=num_neurons,
    input_indices=torch.randint(high=num_neurons, size=(input_size,)),
)

# for now, lets just read outputs from all neurons
output_projection_matrix = None

Downloading...
From (original): https://drive.google.com/uc?id=18448HYpYrm60boziHG73bxN4CK5jG-1g
From (redirected): https://drive.google.com/uc?id=18448HYpYrm60boziHG73bxN4CK5jG-1g&confirm=t&uuid=1c1721af-76e8-4f9c-9e5f-bf15e9e4c66f
To: /net/vast-storage/scratch/vast/mcdermott/lakshmin/hackathon-test/bioplnn/examples/turaga-dros-visual-connectome.pt
100%|█████████████████████████████████████████| 111M/111M [00:01<00:00, 108MB/s]


## Setting up the classifier wrapper

Here, we have written a simple utility that adds an output projecting layer from the connectome to the desired logit space. Again, this is simply an example. Please feel free to add sophistication to this as you please.

In [4]:
model = ConnectomeODEClassifier(
    rnn_kwargs={
        "input_size": input_size,
        "hidden_size": num_neurons,
        "connectivity_hh": torch.abs(connectome),
        "connectivity_ih": input_projection_matrix,
        "output_neurons": output_projection_matrix,
        "nonlinearity": "Sigmoid",
        "batch_first": False,
        "compile_solver_kwargs": {
            "mode": "max-autotune",
            "dynamic": False,
            "fullgraph": True,
        },
    },
    num_classes=10,
    fc_dim=256,
    dropout=0.5,
).to(device)
print(model)

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Define the loss function
criterion = nn.CrossEntropyLoss()

ConnectomeODEClassifier(
  (rnn): ConnectomeODERNN(
    (nonlinearity): Sigmoid()
    (hh): SparseLinear()
    (ih): SparseLinear()
    (solver): OptimizedModule(
      (_orig_mod): AutoDiffAdjoint(step_method=Dopri5(
        (term): ODETerm()
      ), step_size_controller=IntegralController(
        (term): ODETerm()
      ), max_steps=None, backprop_through_step_size_control=True)
    )
  )
  (out_layer): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=47521, out_features=256, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=256, out_features=10, bias=True)
  )
)


In [5]:
# Dataloader setup
transform = T.Compose([T.ToTensor(), T.Normalize((0.1307,), (0.3081,))])
train_data = MNIST(root="data", train=True, transform=transform, download=True)
train_loader = DataLoader(
    train_data, batch_size=128, num_workers=0, shuffle=True
)

In [6]:
n_epochs = 1
# print training statistics every five batches
log_frequency = 50
model = model.train()

## Note: Things to look out for

- Depending on the GPU you have access to and the number of CPUs you will have to adjust `batch_size` and `num_workers` in the DataLoader, as well as `num_steps` in the model forward pass. For reference, on a A100 GPU and a single CPU `batch_size = 256`, `num_workers = 0`, and `num_steps=5` is a reasonable estimate.
- In this example, we have used the torch compiler with the `max-autotune` flag. This means the first few steps in the first epoch WILL BE EXTREMELY SLOW. But, this will dramatically improve as the training goes on. Fret not, early on!

In [7]:
running_loss, running_correct, running_total = 0, 0, 0

for epoch in range(n_epochs):
    for i, (x, labels) in enumerate(train_loader):
        x = x.to(device)
        labels = labels.to(device)
        torch._inductor.cudagraph_mark_step_begin()
        logits = model(x, num_steps=2)
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate running accuracy and loss
        _, predicted = torch.max(logits, 1)
        running_total += labels.size(0)
        running_correct += (predicted == labels).sum().item()
        running_loss += loss.item()

        running_acc = running_correct / running_total
        if (i + 1) % log_frequency == 0:
            print(
                f"Training | Epoch: {epoch} | "
                + f"Loss: {running_loss:.4f} | "
                + f"Acc: {running_acc:.2%}"
            )
            running_loss, running_correct, running_total = 0, 0, 0

Training | Epoch: 0 | Loss: 180.6905 | Acc: 10.23%
Training | Epoch: 0 | Loss: 115.1636 | Acc: 9.80%
Training | Epoch: 0 | Loss: 115.1504 | Acc: 9.77%
Training | Epoch: 0 | Loss: 115.1351 | Acc: 9.72%
Training | Epoch: 0 | Loss: 115.1452 | Acc: 9.11%
Training | Epoch: 0 | Loss: 115.0842 | Acc: 10.80%
Training | Epoch: 0 | Loss: 115.1163 | Acc: 11.39%
Training | Epoch: 0 | Loss: 115.0861 | Acc: 10.94%
Training | Epoch: 0 | Loss: 115.0515 | Acc: 11.91%
